In [ ]:
!mkdir -p /data/sets/nuimages  # Make the directory to store the nuImages dataset in.
!wget https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz  # Download the nuImages mini split.
!tar -xf nuimages-v1.0-mini.tgz -C /data/sets/nuimages  # Uncompress the nuImages mini split.
!pip install lightning numpy nuscenes-devkit pillow torch

In [ ]:
!git clone https://github.com/GordonGustafson/semantic-segmentation.git
%cd semantic-segmentation/

In [ ]:
from dataloader import *
from nuimages import NuImages

from matplotlib import pyplot as plt


nuimages = NuImages(dataroot='/data/sets/nuimages', version='v1.0-mini', verbose=True, lazy=True)
dataset = NuImagesDataset(nuimages)

sem_seg_sample = dataset['0f37924ef2b54da7a233091d95311a38']

fig, axs = plt.subplots(2)
axs[0].imshow(sem_seg_sample.image, interpolation='nearest')
axs[1].imshow(sem_seg_sample.segmentation_mask, interpolation='nearest')
fig.show()